# Test

#### Import the necessary libraries

In [ ]:
from pathlib import Path
import pandas as pd
import sys

# Add the root project directory to the Python path
ROOT = Path.cwd().parent  # This will get the project root since the notebook is in 'notebooks/'
sys.path.append(str(ROOT))
from configs.path_config import EXTRACTED_DATA_DIR, MODEL_DIR, OUTPUT_DIR
from configs.model_config import INPUT_FEATURES, OUTPUT_FEATURES, PARAMS
from src.processing import dataset
from src import utils

#### Create the data loader

In [ ]:
folder_path = EXTRACTED_DATA_DIR / 'group_alvbrodel_shifted'
data = dataset.StrainDataset(
    folder_path, 
    INPUT_FEATURES, 
    OUTPUT_FEATURES, 
    sequence_length=PARAMS['sequence_length'], 
    batch_size=PARAMS['batch_size'],  
    test_size=PARAMS['test_size'], 
    start_idx=0)

#### Load the model

In [ ]:
model_name = 'lstm_model_supportII_20_128_2_0.001_0.3'
model_path = MODEL_DIR / model_name
model, mean_error, std_error = utils.load_model_and_threshold(model_path)
threshold = mean_error + 10 * std_error
print(f"Mean error: {mean_error}")
print(f"Standard deviation of error: {std_error}")
print(f"Threshold: {threshold}")

#### Plot the anomalies

In [ ]:
N = len(data.test_data)

input_feature_names = data.input_feature_names
output_feature_names = data.output_feature_names
print(f"Feature names: {output_feature_names}")
print(f"Timestamps test length: {len(data.timestamps_test)}")
print(f"Sample timestamps: {data.timestamps_test[:5]}")
timestamps = pd.DatetimeIndex(data.timestamps_test[:N])

# save_dir = OUTPUT_DIR / 'figures' / 'RESULTS' / f"{model_name}_reconstruction_plot_sep.pdf"
save_dir = OUTPUT_DIR / 'figures' / 'RESULTS' / f"{model_name}_reconstruction_plot.pdf"
reconstructed = utils.plot_reconstruction(data.test_data, model, N, input_feature_names, output_feature_names, timestamps, threshold, save_dir, mode=3, subplots=False) #put subplots=False to plot all features in separate plots
